In [9]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy
from sklearn.metrics import roc_curve , auc, confusion_matrix
from sklearn.model_selection import train_test_split

In [15]:
from imblearn.over_sampling import SMOTE

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [40]:
# in our case params is the number of estimators
def modelTrain(models, params, trainX, trainY, validX, validY):
    models_auc = []
    for i, model in enumerate(models):
        model.fit(trainX, trainY)
        yPredicted = model.predict_proba(validX)
        fpr, tpr, _ = roc_curve(validY, yPredicted[:,1])
        ar = auc(fpr, tpr)
        models_auc.append(ar)
        print('Model number {}. Number of estimators = {}, AUC = {}'.format(i, params[i], ar))
    return models, models_auc

In [5]:
features = pd.read_csv('modifiedTraining.csv')
labels = pd.read_csv('labels.csv')

In [6]:
features.shape

(15419, 159)

In [7]:
labels.shape

(15419, 1)

In [69]:
trainingX, testX, trainingY, testY = train_test_split(features, labels, random_state=1, test_size=0.2)

In [70]:
trainX, validationX, trainY, validationY = train_test_split(trainingX, trainingY, random_state=1, test_size=0.2) 

In [71]:
sm = SMOTE()

In [72]:
x,y = sm.fit_sample(trainX, trainY.squeeze())

In [73]:
from collections import Counter
Counter(y)

Counter({0.0: 9257, 1.0: 9257})

In [74]:
noOfEstimators = list(range(4,100,2))

In [75]:
models = []
for estimators in noOfEstimators:
    models.append(RandomForestClassifier(n_estimators=estimators, random_state=1))

In [76]:
models, roc_auc = modelTrain(models, noOfEstimators, x, y, validationX, validationY)

Model number 0. Number of estimators = 4, AUC = 0.6410087678359189
Model number 1. Number of estimators = 6, AUC = 0.6726264926789504
Model number 2. Number of estimators = 8, AUC = 0.7015933317154218
Model number 3. Number of estimators = 10, AUC = 0.7256379860735155
Model number 4. Number of estimators = 12, AUC = 0.7404029844244902
Model number 5. Number of estimators = 14, AUC = 0.7649283451876465
Model number 6. Number of estimators = 16, AUC = 0.7629899627258074
Model number 7. Number of estimators = 18, AUC = 0.7773302593014356
Model number 8. Number of estimators = 20, AUC = 0.7705785900523332
Model number 9. Number of estimators = 22, AUC = 0.7730381267073634
Model number 10. Number of estimators = 24, AUC = 0.7720611570556499
Model number 11. Number of estimators = 26, AUC = 0.7729805663934886
Model number 12. Number of estimators = 28, AUC = 0.7815539417925215
Model number 13. Number of estimators = 30, AUC = 0.7881982688346681
Model number 14. Number of estimators = 32, AUC

Maximum AUC on validation set is achieved at number of estimators = 98
So, evaluating Random forest classifier on test set using n_estimators = 98

In [77]:
finalModel = RandomForestClassifier(n_estimators=98, random_state=1)

In [78]:
finalModel.fit(x,y)
yPredicted = finalModel.predict(testX)

In [79]:
confusion_matrix(testY, yPredicted)

array([[2910,    0],
       [ 173,    1]])

In [68]:
(testY == 1).sum()

0.000000000000000000e+00    216
dtype: int64

In [62]:
testY

,0.000000000000000000e+00
11444,0.0
9416,0.0
8076,0.0
12320,0.0
2141,1.0
10615,0.0
11033,0.0
9093,0.0
13994,0.0
6549,0.0


In [46]:
help(confusion_matrix)

Help on function confusion_matrix in module sklearn.metrics.classification:

confusion_matrix(y_true, y_pred, labels=None, sample_weight=None)
    Compute confusion matrix to evaluate the accuracy of a classification
    
    By definition a confusion matrix :math:`C` is such that :math:`C_{i, j}`
    is equal to the number of observations known to be in group :math:`i` but
    predicted to be in group :math:`j`.
    
    Thus in binary classification, the count of true negatives is
    :math:`C_{0,0}`, false negatives is :math:`C_{1,0}`, true positives is
    :math:`C_{1,1}` and false positives is :math:`C_{0,1}`.
    
    Read more in the :ref:`User Guide <confusion_matrix>`.
    
    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (correct) target values.
    
    y_pred : array, shape = [n_samples]
        Estimated targets as returned by a classifier.
    
    labels : array, shape = [n_classes], optional
        List of labels to index the m

In [31]:
help(auc)

Help on function auc in module sklearn.metrics.ranking:

auc(x, y, reorder=False)
    Compute Area Under the Curve (AUC) using the trapezoidal rule
    
    This is a general function, given points on a curve.  For computing the
    area under the ROC-curve, see :func:`roc_auc_score`.  For an alternative
    way to summarize a precision-recall curve, see
    :func:`average_precision_score`.
    
    Parameters
    ----------
    x : array, shape = [n]
        x coordinates.
    y : array, shape = [n]
        y coordinates.
    reorder : boolean, optional (default=False)
        If True, assume that the curve is ascending in the case of ties, as for
        an ROC curve. If the curve is non-ascending, the result will be wrong.
    
    Returns
    -------
    auc : float
    
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn import metrics
    >>> y = np.array([1, 1, 2, 2])
    >>> pred = np.array([0.1, 0.4, 0.35, 0.8])
    >>> fpr, tpr, thresholds = metrics.roc_